In [1]:
%load_ext autoreload
%autoreload 2
    

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/raw/wfh.csv')

In [4]:
df.head()

,id,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,work_home_actual
0,0,5.962247,40K - 60K,2.119485,8.568058,False,Friday,0.212653,1
1,1,0.535872,40K - 60K,2.357199,5.425382,True,Tuesday,4.927549,0
2,2,1.969519,40K - 60K,2.366849,8.247158,False,Monday,0.520817,1
3,3,2.530410,20K - 40K,2.318722,7.944251,False,Tuesday,0.453649,1
4,4,2.253635,60K+,2.221265,8.884478,True,Thursday,5.695263,1


In [5]:
df.shape

(50000, 9)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          50000 non-null  int64  
 1   distance_from_office        50000 non-null  float64
 2   salary_range                50000 non-null  object 
 3   gas_price_per_litre         50000 non-null  float64
 4   public_transportation_cost  50000 non-null  float64
 5   wfh_prev_workday            50000 non-null  bool   
 6   workday                     50000 non-null  object 
 7   tenure                      50000 non-null  float64
 8   work_home_actual            50000 non-null  int64  
dtypes: bool(1), float64(4), int64(2), object(2)
memory usage: 3.1+ MB


In [7]:
df.describe(include='all')

,id,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,work_home_actual
count,50000.000000,50000.000000,50000,50000.000000,50000.000000,50000,50000,50000.000000,50000.000000
unique,NaN,NaN,4,NaN,NaN,2,5,NaN,NaN
top,NaN,NaN,0 - 20K,NaN,NaN,True,Wednesday,NaN,NaN
freq,NaN,NaN,19918,NaN,NaN,27547,10095,NaN,NaN
mean,24999.500000,3.929033,NaN,2.049616,7.323907,NaN,NaN,4.600040,0.499580
std,14433.901067,4.079528,NaN,0.334385,1.630390,NaN,NaN,2.301937,0.500005
min,0.000000,0.002210,NaN,1.400369,4.003417,NaN,NaN,0.002253,0.000000
25%,12499.750000,0.897909,NaN,1.769163,6.109763,NaN,NaN,2.797947,0.000000
50%,24999.500000,2.380855,NaN,2.189073,8.074422,NaN,NaN,5.584845,0.000000
75%,37499.250000,5.679015,NaN,2.337894,8.627489,NaN,NaN,6.531917,1.000000


In [8]:
df_cleaned = df.copy()

In [9]:
y = df_cleaned.pop('work_home_actual')

In [10]:
from sklearn.preprocessing import OrdinalEncoder

In [11]:
ord_enc = OrdinalEncoder(categories=[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']])

In [12]:
df_cleaned['workday'] = ord_enc.fit_transform(df_cleaned[['workday']])

In [13]:
df_cleaned = pd.get_dummies(df_cleaned, columns=["salary_range"])

In [14]:
df_cleaned.drop(["id"], axis=1, inplace=True)

In [15]:
from sklearn.model_selection import train_test_split

In [17]:
features = df_cleaned

In [18]:
X_data, X_test, y_data, y_test = train_test_split(features, y, test_size=0.2, random_state=8)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=8)

In [19]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(32000, 10)
(8000, 10)
(10000, 10)


In [20]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(32000,)
(8000,)
(10000,)


In [21]:
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_val.to_csv('../data/processed/X_val.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [22]:
from sklearn.dummy import DummyClassifier
base_clf = DummyClassifier(strategy='most_frequent')
base_clf.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [23]:
from sklearn.metrics import roc_auc_score
y_proba_preds = base_clf.predict_proba(X_train)
roc_auc_score(y_train, y_proba_preds[:, 1])

np.float64(0.5)